In [ ]:
!pip install pymorphy2

In [ ]:
import pymorphy2
import re

In [ ]:
morph = pymorphy2.MorphAnalyzer()
cats_parsed = morph.parse('котиков')

In [ ]:
cats_parsed[0]

Parse(word='котиков', tag=OpencorporaTag('NOUN,anim,masc plur,accs'), normal_form='котик', score=0.5, methods_stack=((DictionaryAnalyzer(), 'котиков', 2, 9),))

In [ ]:
print(cats_parsed[0].tag.POS) # часть речи
print(cats_parsed[0].tag.case) # падеж
print(cats_parsed[0].tag.number) # число
print(cats_parsed[0].tag.gender) # род

NOUN
accs
plur
masc


**Задание**: реализовать с помощью функционала классов метод, который будет заменять существительное в предложении на существительное переданное в качестве аргумента. Новое существительное должно быть вставлено в том же роде, числе и падеже, что и существительное которое оно заменило, а число, род и падеж других частей речи должны согласовываться с новым словом

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
class Word:
  
  def __init__(self, text: str, parsed: pymorphy2.analyzer.Parse):
    self.text = text
    self.parsed = parsed[0]
    self.pos = self.parsed.tag.POS
    self.number = self.parsed.tag.number

  def inflect_by_noun(self, tag: pymorphy2.tagset.OpencorporaTag):

    gender = tag.gender
    number = tag.number
    case = tag.case
    inflect = None

    if self.pos == 'NOUN':
      inflect = self.parsed.inflect({case, number})
    elif self.pos in {'ADJF', 'ADJS', 'PRTF', 'PRTS'} and self.number == 'sing':
      inflect = self.parsed.inflect({gender})
    elif self.pos == 'VERB' and self.number == 'sing' and self.parsed.tag.tense == 'past':
      inflect = self.parsed.inflect({gender})
    if not inflect:
      return self.text
    return inflect.word

In [ ]:
class Sentence:

  def __init__(self, text: str, analyzer: pymorphy2.MorphAnalyzer):
    self.text = text
    self.analyzer = analyzer
    self.words = re.findall(r'[\w\d]+', text)
    self.parsed_words = self.parse_words()

  def parse_words(self):
    res = []
    for word in self.words:
      parsed_word = self.analyzer.parse(word)
      res.append(Word(word, parsed_word))
    return res

  def replace_nouns(self, noun: str):
    res = []
    parsed_noun = self.analyzer.parse(noun)
    noun = Word(noun, parsed_noun)
    for word in self.parsed_words:
      if word.pos == 'NOUN':
        res.append(noun.inflect_by_noun(word.parsed.tag))
      else:
        res.append(word.inflect_by_noun(noun.parsed.tag))
    return ' '.join(res).capitalize()
    

In [ ]:
s = Sentence('А красивая роза упала на лапу Азора', morph)
s.replace_nouns('Телевизор')

'А красивый телевизор упал на телевизор телевизора'

In [ ]:
s = Sentence('Пушистая мама стирала мокрую раму', morph)
s.replace_nouns('котик')

'Пушистый котик стирал мокрого котика'

**Задание**:реализовать класс LinSpace, который будет создавать последовательность из равноудаленных точек на отрезке $[start, end]$. На вход конструктору должны подаваться 2 числа: $start, end$, которые обозначают начало и конец отрезка. Экземпляр класса должен поддерживать следующий функционал: 
* `d[n]` — последовательность из $n$ равноудалённых точек от начала до конца отрезка (включая конец). 
* `d[i:n]` — $i$-я точка такой последовательности. 
* `d[i:j:n]` — последовательность точек, начиная с $i$-той и заканчивая $j-1$-й точкой исходной последовательности

In [ ]:
class LinSpace:

  def __init__(self, start: int, end: int):
    self.start = start
    self.end = end
    self.step = 0

  def __getitem__(self, item):
    if isinstance(item, slice):
      if item.start is None and item.stop is None:
        self.step = (self.end - self.start) / (item.step - 1)
        return self.gen(self.start, item.step, self.step)
      elif item.start is None:
        self.step = (self.end - self.start) / (item.step - 1)
        return self.gen(self.start, item.stop, self.step)
      elif item.stop is None:
        self.step = (self.end - self.start) / (item.step - 1)
        return self.gen(item.start, item.step, self.step)
      elif item.step is None:
        self.step = (self.end - self.start) / (item.stop - 1)
        return self.start + item.start * self.step
      else:
        self.step = (self.end - self.start) / (item.step - 1)
        return self.gen(item.start, item.stop, self.step)
    else:
      self.step = (self.end - self.start) / (item - 1)
      return self.gen(self.start, item, self.step)

  def gen(self, start, end, step):
    for i in range(start, end):
      yield self.start + i * step

In [ ]:
ls = LinSpace(0, 8)
print(*ls[5])

0.0 2.0 4.0 6.0 8.0


In [ ]:
print(*ls[::5])

0.0 2.0 4.0 6.0 8.0


In [ ]:
print(*ls[:2:5])

0.0 2.0


In [ ]:
print(*ls[3::5])

6.0 8.0


In [ ]:
print(ls[1:5])

2.0


In [ ]:
print(*ls[1:3:5])

2.0 4.0


In [ ]:
ls = LinSpace(0, 10)
ls[5]
ls[5:10]
ls[2:5:10]
ls[:5:10]
ls[2::13]

<generator object LinSpace.gen at 0x7f296851aa50>